In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta, norm, invgamma

np.random.seed(111)

# Binomial 
n = 10
p = 0.5
bin_alphas = [1, 2, 10]
bin_betas = [1, 4, 20]

# Gaussian
mu_true = 2.0
sigma_true = 1.0

mu_0s = [1.5, 2.0, 2.5]
tau_0s = [1.0, 1.01, 0.99]

gauss_alphas = [1, 4, 20]
gauss_betas = [1, 4, 20]


n_samples = 100

binomial_data = np.random.binomial(n, p, n_samples)
gaussian_data = np.random.normal(mu_true, sigma_true, n_samples)


In [3]:
def get_mse(estimates, true_value):
    mse = [(est - true_value)**2 for est in estimates]
    return mse

# Plot Posterior Density
def plot_posterior_density(prior, posterior, label, x_range, n_points=100):
    x = np.linspace(x_range[0], x_range[1], n_points)
    plt.plot(x, prior.pdf(x), label=f'Prior {label}')
    plt.plot(x, posterior.pdf(x), label=f'Posterior {label}')


In [2]:
def generate_binomial_data(n, p, size):
    return np.random.binomial(n, p, size)

def binomial_mle(data, n):
    return np.mean(data) / n

# Beta Mean: alpha + num_successes / alpha + beta + total_trials

def binomial_conjugate(data, n, alpha, beta):
    num_successes = np.sum(data) 
    num_trials = n * len(data)
    return (alpha + num_successes) / (alpha + beta + num_trials)